# Deep Graph Library(DGL) 基础

Deep Graph Library (DGL)是一个简单通用的图深度学习的库，可以使用PyTorch，TensorFlow和MXNet作为后端框架。利用DGL，可以轻松编写和训练图神经网络，并用于与结构化数据相关的应用。DGL支持许多与图数据相关的功能，比如异质图、点云、小批量加载器和各种常用基准数据集等等。

## DGL的安装

现在DGL支持多种安装方法，见官网https://www.dgl.ai/pages/start.html 。我们推荐使用pip来安装DGL。

注意：正确选择对应的计算机系统、CUDA或者CPU版本。

`pip install dgl -f https://data.dgl.ai/wheels/repo.html`

建议安装DGL最新版，当前是0.9版本，不过上面的命令会安装最新的版本；PyTorch建议1.10.0或更高版本。

`pip install torch==1.10.0`

## DGL图的基本概念和操作

实际业务中要处理的数据绝大部分是异构图。这里我们用下面这张图的异构图来作为例子演示如何构建异构的DGL图。

<img src='heterogeneous_graph.png' width=40%>

该图中包含了公司和产品这两种点，以及制造、下游、供应，一共3种关系。

下面，我们将演示如何从数据中构建异质图。首先，我们从csv文件中读取数据，包括公司（node）信息和产品（node）信息，以及3种边（制造、下游、供应）的信息。

In [1]:
import pandas as pd
import torch as th
import dgl
import numpy as np

df_company     = pd.read_csv('raw_data/companies.csv')
df_product     = pd.read_csv('raw_data/products.csv')
df_produce     = pd.read_csv('raw_data/comp_produce_prod.csv')
df_downstream  = pd.read_csv('raw_data/prod_downstream_prod.csv')
df_supply      = pd.read_csv('raw_data/comp_supply_comp.csv')
print(df_company.keys())
print(df_product.keys())

/home/studio-lab-user/.conda/envs/dgl/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Index(['CID', 'size'], dtype='object')
Index(['PID', 'price'], dtype='object')


In [2]:
df_downstream.head()

,PID,PID.1
0,prod1_1,prod1_2
1,prod1_3,prod2_1
2,prod2_1,prod2_2
3,prod2_2,prod1_2
4,prod2_2,prod3_1


### 将节点编号从0重排

在DGL里，我们一般需要对对Node ID从0开始重新排序以满足DGL数据存储的特点。注意这里，公司和产品都从0开始设置ID。

In [3]:
company_mapping = {idx: i for i, idx in enumerate(df_company['CID'].unique())}
num_companies = len(company_mapping)

product_mapping = {idx: i for i, idx in enumerate(df_product['PID'].unique())}
num_products = len(product_mapping)

### 根据边列表创建图结构

下面，我们创建边列表，由于图中包含制造、下游、供应3种边，为了进行区分，我们需要对每一种边都创建单独的列表，因此我们用字典来存储。

In [4]:
edge_dict = {}

# produce relation and its reversed edges
src = [company_mapping[idx] for idx in df_produce['CID']] # 起始节点
dst = [product_mapping[idx] for idx in df_produce['PID']] # 终止节点
edge_dict['produce'] =  (th.tensor(src), th.tensor(dst))
edge_dict['producedby'] =  (th.tensor(dst), th.tensor(src))

# downstream relation and its reversed edges
src = [product_mapping[idx] for idx in df_downstream['PID']] 
dst = [product_mapping[idx] for idx in df_downstream['PID.1']] 
edge_dict['downstream'] =  (th.tensor(src), th.tensor(dst))
edge_dict['upstream'] =  (th.tensor(dst), th.tensor(src))
          
# supply relation and its reversed edges
src = [company_mapping[idx] for idx in df_supply['CID']] 
dst = [company_mapping[idx] for idx in df_supply['CID']] 
edge_dict['supply'] =  (th.tensor(src), th.tensor(dst))
edge_dict['suppliedby'] =  (th.tensor(dst), th.tensor(src))

DGL中，我们用dgl.heterograph(）建立异质图，其输入为包含边类型和边数据的字典。

In [5]:
graph_data = {
   ('company', 'produce', 'product'): edge_dict['produce'],
   ('product', 'downstream', 'product'): edge_dict['downstream'],
   ('company', 'supply', 'company'): edge_dict['supply'],
   ('product', 'producedby', 'company'): edge_dict['producedby'],
   ('product', 'upstream', 'product'): edge_dict['upstream'],
   ('company', 'suppliedby', 'company'): edge_dict['suppliedby']    
}
g = dgl.heterograph(graph_data)
print(g)

Graph(num_nodes={'company': 3, 'product': 8},
      num_edges={('company', 'produce', 'product'): 8, ('company', 'suppliedby', 'company'): 3, ('company', 'supply', 'company'): 3, ('product', 'downstream', 'product'): 7, ('product', 'producedby', 'company'): 8, ('product', 'upstream', 'product'): 7},
      metagraph=[('company', 'product', 'produce'), ('company', 'company', 'suppliedby'), ('company', 'company', 'supply'), ('product', 'product', 'downstream'), ('product', 'product', 'upstream'), ('product', 'company', 'producedby')])


In [6]:
g.edges('all', etype='downstream')

(tensor([0, 2, 3, 4, 4, 5, 6]),
 tensor([1, 3, 4, 1, 6, 4, 7]),
 tensor([0, 1, 2, 3, 4, 5, 6]))

In [7]:
g.edges('all', etype='upstream')

(tensor([1, 3, 4, 1, 6, 4, 7]),
 tensor([0, 2, 3, 4, 4, 5, 6]),
 tensor([0, 1, 2, 3, 4, 5, 6]))

### 保存图

In [8]:
from dgl.data.utils import save_graphs, load_graphs

hetero_local_path = 'hetero_dgl.bin'
save_graphs(hetero_local_path, [g])